In [1]:
%reload_ext autoreload
%autoreload 2

# Imports
Import the required classes and functions.

In [2]:
from mleko.dataset.convert import CSVToVaexConverter
from mleko.dataset.feature_select import MissingRateFeatureSelector, VarianceFeatureSelector, CompositeFeatureSelector, PearsonCorrelationFeatureSelector
from mleko.dataset.ingest import KaggleIngester
from mleko.dataset.split import ExpressionSplitter, RandomSplitter
from mleko.pipeline import Pipeline
from mleko.pipeline.steps import ConvertStep, FeatureSelectStep, IngestStep, SplitStep

# Constants
Define configuration variables.

In [3]:
OWNER_SLUG = 'mlg-ulb'
DATASET_SLUG = 'creditcardfraud'
DATASET_NAME = f'{OWNER_SLUG}/{DATASET_SLUG}'

TARGET_FEATURE = "Class"
TIME_FEATURE = "Time"
META_FEATURES = [TIME_FEATURE, TARGET_FEATURE]
RANDOM_STATE = 1337

# Pipeline Setup

In [27]:
kaggle_data_source = KaggleIngester(
    output_directory=f"data/{DATASET_NAME}/raw", owner_slug=OWNER_SLUG, dataset_slug=DATASET_SLUG
)
csv_to_arrow_converter = CSVToVaexConverter(
    output_directory=f"data/{DATASET_NAME}/converted", downcast_float=True, random_state=RANDOM_STATE
)
random_data_splitter = RandomSplitter(
    cache_directory=f"data/{DATASET_NAME}/split",
    data_split=(0.80, 0.20),
    shuffle=True,
    stratify=TARGET_FEATURE,
    random_state=RANDOM_STATE,
)
expression_data_splitter = ExpressionSplitter(cache_directory=f"data/{DATASET_NAME}/split", expression="Time > 100")
composite_feature_selector = CompositeFeatureSelector(
    cache_directory=f"data/{DATASET_NAME}/feature_selection",
    feature_selectors=[
        MissingRateFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_selection",
            missing_rate_threshold=0.7,
            ignore_features=META_FEATURES,
        ),
        VarianceFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_selection",
            variance_threshold=0.00,
            ignore_features=META_FEATURES
        ),
        PearsonCorrelationFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_selection",
            correlation_threshold=0.7,
            ignore_features=META_FEATURES
        )
    ],
)


pipeline = Pipeline(
    steps=[
        IngestStep(kaggle_data_source, outputs=["raw_csv"]),
        ConvertStep(csv_to_arrow_converter, inputs=["raw_csv"], outputs=["df_clean"]),
        SplitStep(random_data_splitter, inputs=["df_clean"], outputs=["df_train_validate", "df_test"]),
        SplitStep(expression_data_splitter, inputs=["df_train_validate"], outputs=["df_train", "df_validate"]),
        FeatureSelectStep(
            composite_feature_selector,
            inputs=["df_train"],
            outputs=["df_train_features_selected"],
        ),
    ]
)


[2023-05-31 22:28:59] [INFO] Attempting to fetch Kaggle API credentials from environment variables 'KAGGLE_USERNAME' and 'KAGGLE_KEY'. (kaggle_ingester.py:74)
[2023-05-31 22:28:59] [WARNING] Kaggle API credentials not found in environment variables, attempting to fetch from fallback path at ~/.kaggle/kaggle.json. (kaggle_ingester.py:82)
[2023-05-31 22:28:59] [INFO] Kaggle credentials successfully fetched. (kaggle_ingester.py:91)


# Run Pipeline

In [28]:
data_container =  pipeline.run(force_recompute=True).data

[2023-05-31 22:29:06] [INFO] No data container provided. Creating an empty one. (pipeline.py:77)
[2023-05-31 22:29:06] [INFO] Executing step 1: IngestStep. (pipeline.py:81)
[2023-05-31 22:29:06] [INFO] Force Cache Refresh: Downloading mlg-ulb/creditcardfraud/* to data/mlg-ulb/creditcardfraud/raw from Kaggle. (kaggle_ingester.py:287)


[2023-05-31 22:29:11] [INFO] Finished downloading 1 files from Kaggle. (kaggle_ingester.py:303)
[2023-05-31 22:29:11] [INFO] Finished step 1 execution. (pipeline.py:83)
[2023-05-31 22:29:11] [INFO] Executing step 2: ConvertStep. (pipeline.py:81)


[2023-05-31 22:29:12] [INFO] Force Cache Refresh (LRUCache) CSVToVaexConverter.convert: Executing method. (cache_mixin.py:133)


Writing DataFrame to Arrow file: 100%|██████████| 100/100 [00:00<00:00, 116.63it/s]

[2023-05-31 22:29:17] [INFO] Finished step 2 execution. (pipeline.py:83)
[2023-05-31 22:29:17] [INFO] Executing step 3: SplitStep. (pipeline.py:81)
[2023-05-31 22:29:17] [INFO] Force Cache Refresh (LRUCache) RandomSplitter.split: Executing method. (cache_mixin.py:133)
[2023-05-31 22:29:17] [INFO] Shuffling data before splitting. (random_splitter.py:122)
[2023-05-31 22:29:17] [INFO] Splitting data with stratification on column 'Class'. (random_splitter.py:126)


...


Calculating variance for features: 100%|██████████| 29/29 [00:01<00:00, 27.10it/s]

[2023-05-31 22:29:20] [INFO] Dropping features with normalized variance <= 0.0: set(). (variance_feature_selector.py:114)
[2023-05-31 22:29:20] [INFO] Selecting features from the following set: ['Amount', 'V1', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V2', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']. (pearson_correlation_feature_selector.py:44)


[2023-05-31 22:29:20] [INFO] Dropping features with correlation >= 0.7: set(). (pearson_correlation_feature_selector.py:90)


Writing DataFrame to Arrow file: 100%|██████████| 100/100 [00:00<00:00, 865.50it/s]

[2023-05-31 22:29:20] [INFO] Finished step 5 execution. (pipeline.py:83)


In [34]:
import vaex
from mleko.dataset.feature_select import PearsonCorrelationFeatureSelector
from mleko.utils.vaex_helpers import get_column
df = vaex.from_arrays(
     a=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
     b=[1, 2, 3, 4, 5, 6, 7, 8, 9, 9],
     c=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 )
feature_selector = PearsonCorrelationFeatureSelector(
    cache_directory=".",
    correlation_threshold=0.75,
)
selected_features = feature_selector.select_features(df)
selected_features.get_column_names()

[2023-05-31 22:52:06] [INFO] Cache Hit (LRUCache) PearsonCorrelationFeatureSelector.select_features: Using cached output. (cache_mixin.py:124)


['a', 'c']